###

Pedro Joás Freitas Lima - 548292

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Funções básicas 

Normaliza  e divide_treino_teste

In [2]:
def normaliza(array):
    mean = np.average(array)
    std = np.std(array)

    array_padronizado = (array - mean)/std

    return array_padronizado

In [252]:
def divide_treino_teste(x, y, tamanho_treino=0.8):

    n = x.shape[0]
    q_treino = int(n * tamanho_treino)
    
    rng = np.random.default_rng()
    indices = rng.permutation(n)
    idx_treino = indices[:q_treino]
    idx_teste = indices[q_treino:]
    
    x_treino = x[idx_treino]
    x_teste = x[idx_teste]
    y_treino = y[idx_treino]
    y_teste = y[idx_teste]
    
    x_treino = np.concatenate([np.ones((q_treino, 1)), x_treino], axis=1)
    x_teste = np.concatenate([np.ones((n - q_treino, 1)), x_teste], axis=1)
    
    return x_treino, x_teste, y_treino, y_teste

## Funções de predição e custos

sigmoide, cross_entropy_loss e multi_cross_entropy_loss

In [ ]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

def cross_entropy_loss(y_treino, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_treino*np.log(y_pred) + (1-y_treino)*np.log(1-y_pred))

def multi_cross_entropy_loss(y_treino, y_pred):
    epsilon = 1e-15  # para evitar log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -np.sum(y_treino * np.log(y_pred)) / y_treino.shape[0]
    return loss


## Implementações dos modelos

Regressao Logistica, Analise do discriminante gaussiano, naivy bayes gaussiano e regressão softmax

In [315]:
class RegressaoLogistica:
    def __init__(self):
        self.pesos = []
        
    def treinar(self, x_treino, y_treino, r=0, passo_aprendizado=0.1, epochs=200):
        self.cross_entropy_history = []

        n = x_treino.shape[0]
        self.pesos = np.zeros(x_treino.shape[1])
        
        t = 0
        while t < epochs:
            t+=1
            y_pred = sigmoide(x_treino @ self.pesos)
            erro = (y_treino - y_pred)
            self.cross_entropy_history.append(cross_entropy_loss(y_treino, y_pred))

            gradientes = (1/n) * x_treino.T @ erro
            gradientes[1:] -= r * self.pesos[1:]    
                    
            self.pesos += passo_aprendizado * gradientes
            
    
    def predizer(self, x_teste):
        y_pred = sigmoide(x_teste @ self.pesos)
        
        return np.where(y_pred > 0.5, 1, 0)
        
    
    def hist_cross_entropy(self):
        sns.lineplot(self.cross_entropy_history)
        plt.title('Entropia cruzada ao longo das iterações do GD')
        plt.xlabel('Epochs')
        plt.ylabel('Cross-entropy')
        plt.show()
    


In [523]:
class AnaliseDiscriminanteGaussiano:
    
    def __init__(self):
        pass
    
    def treinar(self, x_treino, y_treino):
        self.classes = np.unique(y_treino)
        self.priori = 1/self.classes.shape[0]
        self.classes_parametros = {}
        for classe in self.classes:
            treino_classe = x_treino[np.where(y_treino == classe)[0]]
            matriz_covariancia = np.cov(treino_classe, rowvar=False)
            vetor_medias = np.mean(treino_classe, axis=0)

            self.classes_parametros[classe] = (vetor_medias, matriz_covariancia)


    def __calcula_prob(self, x_teste_amostra):
        probs = []
        for classe in self.classes:
            vetor_medias, matriz_covariancia = self.classes_parametros[classe]

            diferenca = x_teste_amostra - vetor_medias
            determinante = np.linalg.det(matriz_covariancia) + 1e-9

            inversa = np.linalg.pinv(matriz_covariancia)
            priori = np.log(self.priori)
            
            prob_classe = -(1/2)*np.log(determinante) - (1/2)*diferenca.T @ inversa @ diferenca + priori

            probs.append(prob_classe)
        
        return self.classes[np.argmax(probs)]
            
    def predizer(self, x_teste):
        return np.apply_along_axis(self.__calcula_prob, axis=1, arr=x_teste)



In [445]:
class NaivyBayesGaussiano:
    def __init__(self):
        pass
    
    def treinar(self, x_treino, y_treino):
        self.classes = np.unique(y_treino)
        self.priori = 1/self.classes.shape[0]
        self.classes_parametros = {}
        for classe in self.classes:
            treino_classe = x_treino[np.where(y_treino == classe)[0]]
            variancias = np.var(treino_classe, axis=0)
            vetor_medias = np.mean(treino_classe, axis=0)

            self.classes_parametros[classe] = (vetor_medias, variancias)
    
    def __calcula_prob(self, x_teste_amostra):
        probs = []
        for classe in self.classes:
            vetor_medias, variancias = self.classes_parametros[classe]

            diferenca = (x_teste_amostra - vetor_medias)**2

            priori = np.log(self.priori)
            
            prob_classe = priori - 0.5*np.sum(np.log(2*np.pi*variancias + 1e-9)) - 0.5*np.sum(diferenca/(variancias + 1e-9))

            probs.append(prob_classe)
        
        return self.classes[np.argmax(probs)]
    
    def predizer(self, x_teste):
        return np.apply_along_axis(self.__calcula_prob, axis=1, arr=x_teste)

In [790]:
class RegressaoSoftmax:
    def __init__(self):
        pass

    def treinar(self, x_treino, y_treino, r=0, passo_aprendizado=0.1, epochs=200):
        y_treino = self.__one_hot_encoding(y_treino)
        
        self.multi_cross_entropy_history = []
        n = x_treino.shape[0]
        self.qtd_classes = y_treino.shape[1]
        self.pesos = np.zeros((x_treino.shape[1], self.qtd_classes))
        
        self.t = 0
        while self.t < epochs:
            self.t+=1
            y_pred = self.softmax(x_treino) # = n x k
            erro = (y_treino - y_pred) # erro é uma matriz n x k
            self.multi_cross_entropy_history.append(multi_cross_entropy_loss(y_treino, y_pred))
            
            gradientes = (1/n) * x_treino.T @ erro # (d x n) x (n x k) = (d x k)
            gradientes[1:] += r * self.pesos[1:]

            self.pesos += passo_aprendizado * gradientes
        

    def predizer(self, x_teste):
        y_pred = self.softmax(x_teste)
        
        y_pred = np.argmax(y_pred, axis=1)
        return y_pred
    def softmax(self, x):
        numerador = np.exp(x @ self.pesos)
        denominador = np.sum(numerador, axis=1)
        resultado = numerador.T/denominador
        return resultado.T
    
    def hist_multi_cross_entropy(self):
        sns.lineplot(self.multi_cross_entropy_history)
        plt.title('Múltipla entropia cruzada ao longo das iterações do GD')
        plt.xlabel('Epochs')
        plt.ylabel('Multi Cross-entropy')
        plt.show()

    def __one_hot_encoding(self,labels):
        qtd_labels = np.unique(labels).size
        codificado = np.zeros((labels.shape[0], qtd_labels))

        for i, label in enumerate(labels):
            codificado[i][int(label)] = 1
        
        return codificado

## Métricas de avaliação

acurácia média global, desvio padrão global, acurácia média por classe e desvio padrão por classe

In [525]:
def acuracia_global(y_verdadeiro, y_pred):
    resultado = np.where(y_pred == y_verdadeiro, 1, 0)
    acertos = np.count_nonzero(resultado)
    acuracia = (acertos/resultado.shape[0])
    return acuracia

def acuracia_por_classe(y_verdadeiro, y_pred):
    acuracias = []
    for c in np.unique(y_pred):
        resultados = np.where(y_pred == y_verdadeiro, 1, 0)
        indexes = np.where(y_pred == c)[0]
        acuracia_classe = np.count_nonzero(resultados[indexes])/indexes.shape[0]
        acuracias.append(acuracia_classe)

    return acuracias

## Implementa KFolds

In [801]:
def kfolds(x, y, models:dict, k=10):
    dados_treino = np.hstack([x, y.reshape(-1,1)])
    np.random.shuffle(dados_treino)

    k_partes = np.array_split(dados_treino, k)
    metricas = {}
    for nome_modelo, modelo in models.items():
        acuracias_global = []
        acuracias_classes = []
        for i in range(k):
            teste = k_partes[i]
            treino = np.vstack(k_partes[:i] + k_partes[i+1:])

            x_treino,y_treino = treino[:, :-1], treino[:, -1]
            x_teste, y_teste = teste[:, :-1], teste[:, -1]

            modelo.treinar(x_treino, y_treino)
            y_pred = modelo.predizer(x_teste)

            acuracias_global.append(acuracia_global(y_teste, y_pred))
            acuracias_classes.append(acuracia_por_classe(y_teste, y_pred))

        metricas[nome_modelo] = [{'Global':(np.mean(acuracias_global), np.std(acuracias_global)), 
                                  'Classes': (np.mean(acuracias_classes, axis=0), np.std(acuracias_classes, axis=0))}]


    
    return metricas

##### Função útil para printar bonito na tela

In [805]:
def organiza_texto(info:dict):
    for k,v in info.items():
        info_global = v[0]['Global']
        info_classes = v[0]['Classes']
        print(f'{k}:')
        print(f'Acuracia média global: {info_global[0]}')
        print(f'Desvio padrão global: {info_global[1]}')
        print(f'Acuracia média por classe: {info_classes[0]}')
        print(f'Desvio padrão por classe: {info_classes[1]}')
        print('-'*30)

# Questão 1

In [807]:
ds_cancer = pd.read_csv('breastcancer.csv',header=None)

ds_cancer.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1.0


In [808]:
x = ds_cancer.iloc[:, :-1].apply(normaliza).to_numpy()
y = ds_cancer.iloc[:, -1].to_numpy()

x = np.column_stack([np.ones(x.shape[0]), x])

In [809]:
x_treino, x_teste, y_treino, y_teste = divide_treino_teste(x,y)

In [810]:
reglog = RegressaoLogistica()
adg = AnaliseDiscriminanteGaussiano()
nbg = NaivyBayesGaussiano()

models = {
    'RegressaoLogistica':reglog,
    'AnaliseDiscriminanteGaussiano': adg,
    'NaivyBayesGaussiano': nbg
}

info = kfolds(x_treino, y_treino, models, k=10)

In [811]:
organiza_texto(info)

RegressaoLogistica:
Acuracia média global: 0.9823671497584542
Desvio padrão global: 0.013286780735868307
Acuracia média por classe: [0.97431578 0.99411765]
Desvio padrão por classe: [0.02283726 0.01764706]
------------------------------
AnaliseDiscriminanteGaussiano:
Acuracia média global: 0.8834782608695653
Desvio padrão global: 0.0454695886118347
Acuracia média por classe: [0.98944444 0.77373041]
Desvio padrão por classe: [0.02114763 0.07848394]
------------------------------
NaivyBayesGaussiano:
Acuracia média global: 0.9297101449275363
Desvio padrão global: 0.03343696695541595
Acuracia média por classe: [0.9347427 0.9245285]
Desvio padrão por classe: [0.03887364 0.07358882]
------------------------------


# Questão 2

In [580]:
vehicle = pd.read_csv('vehicle.csv', header=None)

vehicle.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,95.0,48.0,83.0,178.0,72.0,10.0,162.0,42.0,20.0,159.0,176.0,379.0,184.0,70.0,6.0,16.0,187.0,197.0,3.0
1,91.0,41.0,84.0,141.0,57.0,9.0,149.0,45.0,19.0,143.0,170.0,330.0,158.0,72.0,9.0,14.0,189.0,199.0,3.0
2,104.0,50.0,106.0,209.0,66.0,10.0,207.0,32.0,23.0,158.0,223.0,635.0,220.0,73.0,14.0,9.0,188.0,196.0,2.0
3,93.0,41.0,82.0,159.0,63.0,9.0,144.0,46.0,19.0,143.0,160.0,309.0,127.0,63.0,6.0,10.0,199.0,207.0,3.0
4,85.0,44.0,70.0,205.0,103.0,52.0,149.0,45.0,19.0,144.0,241.0,325.0,188.0,127.0,9.0,11.0,180.0,183.0,0.0


In [789]:
x = vehicle.iloc[:, :-1].apply(normaliza).to_numpy()
y = vehicle.iloc[:, -1].to_numpy()

x = np.column_stack([np.ones(x.shape[0]), x])

In [791]:
x_treino, x_teste, y_treino, y_teste = divide_treino_teste(x, y)

In [792]:
reg = RegressaoSoftmax()

reg.treinar(x_treino, y_treino)

In [802]:
regsoft = RegressaoSoftmax()
adg = AnaliseDiscriminanteGaussiano()
nbg = NaivyBayesGaussiano()

models = {
    'RegressaoSoftmax':regsoft,
    'AnaliseDiscriminanteGaussiano': adg,
    'NaivyBayesGaussiano': nbg
}

info = kfolds(x_treino, y_treino, models, k=10)

In [806]:
organiza_texto(info)

RegressaoSoftmax:
Acuracia média global: 0.7156716417910448
Desvio padrão global: 0.07868676859074657
Acuracia média por classe: [0.78504575 0.61061732 0.62256008 0.78552562]
Desvio padrão por classe: [0.07426908 0.14049708 0.14599902 0.13817837]
------------------------------
AnaliseDiscriminanteGaussiano:
Acuracia média global: 0.8415496049165935
Desvio padrão global: 0.06342785721004503
Acuracia média por classe: [0.97784258 0.6888863  0.77554403 0.91879085]
Desvio padrão por classe: [0.02737627 0.14451663 0.10491974 0.08463223]
------------------------------
NaivyBayesGaussiano:
Acuracia média global: 0.43294556628621594
Desvio padrão global: 0.09917448741803685
Acuracia média por classe: [0.61238095 0.47702208 0.47744006 0.3902201 ]
Desvio padrão por classe: [0.33601101 0.20316331 0.11779219 0.10921023]
------------------------------
